# Tesisquare Dataset Analysis

## A data science project by Porsche GTRR

We are the Porsche GTRR a team formed by Grosso Luca, Torterolo Francesco, Risso Beatrice e Robresco Simone

# Tesisquare Dataset Analysis Project

### Roles:


- __Graphics Designer:__ Luca Grosso
- __Coders:__ Simone Robresco, Francesco Torterolo, Beatrice Risso
- __Code reviewer:__ Luca Grosso, Beatrice Risso
- __Notebook Redactor:__ Beatrice Risso, Luca Grosso

## Project Structure

1. __Introduction:__ Overview of the project goals and team roles.
2. __Dataset Description:__ Summary of the dataset provided by Tesisquare and its key features.
3. __Exploratory Data Analysis (EDA):__ High-level exploration of trends and patterns.
4. __Conclusions:__ Insights derived from the analysis.

## CSV Structure

- __SERVICETYPE:__  type of service or vehicle used for the delivery
- __VEHICLETYPE:__ type of vehicle
- __DEPARTURE_COUNTRY:__ country of departure
- __DEPARTURE_ZIPCODE:__ zipocde (CAP) of departure
- __ARRIVAL_COUNTRY:__ country of arrival
- __ARRIVAL_ZIPCODE:__ zipcode (CAP) of arrival

## Introduction

In this project, our team analyzed a dataset provided by the company Tesisquare related to logistics to analyze the data and the relationships between the different features of the dataset. Specifically, we examined the connections between: 


Simone Robresco, Francesco Torterolo, and Beatrice Risso developed the code for data analysis and visualization. Simone Robresco and Luca Grosso designed the types of charts used and the website. Beatrice Risso and Luca Grosso organized the notebook and customized the project presentation.

# Dataset Description

The dataset provides logistics data from a company that handles intercontinental shipments, including:

- Logistics details (e.g., mode of transport used, services utilized, shipping day and date). 
- Travel-related information (e.g., departure and arrival countries, departure and arrival zip codes, distance in km).
- Information related to the transported products (e.g., net and gross weight, volume). 

Understanding the data's structure and cleaning it is crucial before diving into the analysis.

# Exploratory Data Analysis (EDA)

## __Goals of the Analysis__

The main objective of this analysis is to understand the relationships between the characteristics of the shipments and the delivery times. Key areas of interest include:

- The difference in vehicles and services.
- The departure and arrival countries
- The shipping date.


### Importation of libraries and upload of the dataset

In this cell we are importing all the libraries that will be necessary for all the different snippets,
and than upload the dataset via an xls file that we gained before using the API on the original dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import requests
from math import radians, sin, cos, sqrt, atan2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Upload of the dataset
file_path = './delivery_data.xls'
data = pd.read_csv(file_path)

### Basic analysis of the features and values of the dataset

In the next three cells we analyzed the dataset to understand how to work on it, specially trying to understand
how many NULL data for each feature we had

In [ ]:
# Shows the first rows of the dataset
print("First rows of the dataset:")
print(data.head())

In [ ]:
# Descriptive statistics
print("\nDescriptive statistics:")
data.describe()


In [ ]:
print("Dataset size:", data.shape)  # Dataset's dimension
data.info()  # General information about data types and counting NON-null values

# We calculate the number of missing values for each column
missing_values = data.isnull().sum()

# Printing of missing data in different columns 
print("Columns with missing data")
missing_values[missing_values > 0]  # We only show columns with missing values


# List of specific columns to analyze
colonne_selezionate = [
    'SERVICETYPE',
    'VEHICLETYPE',
    'DEPARTURE_COUNTRY',
    'ARRIVAL_COUNTRY',
    'GROSS_WEIGHT_KG',
    'NET_WEIGHT_KG',
    'VOLUME_M3',
    'DECLARED_DISTANCE_KM',
    'DELIVERY_TIME_HH',
    'WDAY'
]

# Displaying the count for each selected column
for column in colonne_selezionate:
    print(f"\nCount values for the column '{column}':")
    print(data[column].value_counts())
    print("-" * 50)  # Separation line for better readability
    

### Count of shippings arrival and departure

Than we tried to find out how many deliveries we had from various combination of start point and arrival point:

- from US to Everywhere
- from US to Everywhere (listed based on the country of arrival)
- from US to US
- from US to IT
- from IT to Everywhere
- from IT to Everywhere (listed based on the country of arrival)
- from IT to US
- from IT to IT

In [ ]:
# Count shipments from the United States (US)
us_shipments_count = data[data['DEPARTURE_COUNTRY'] == 'US'].shape[0]
print(f"\nNumber of shipments from US: {us_shipments_count}")


# Shipment count by country of arrival, only for those departing from the US
arrival_counts_us = data[data['DEPARTURE_COUNTRY'] == 'US']['ARRIVAL_COUNTRY'].value_counts()
print("\nShipment count by country of arrival (shipments from US only):")
print(arrival_counts_us)


# Total shipments from US to IT
us_to_it_count = data[(data['DEPARTURE_COUNTRY'] == 'US') & (data['ARRIVAL_COUNTRY'] == 'IT')].shape[0]
print(f"Number of shipments from US to IT: {us_to_it_count}")


# Total shipments from US to US
us_to_us_count = data[(data['DEPARTURE_COUNTRY'] == 'US') & (data['ARRIVAL_COUNTRY'] == 'US')].shape[0]
print(f"Number of shipments from US to US: {us_to_us_count}")


# Count shipments from Italy (IT)
it_shipments_count = data[data['DEPARTURE_COUNTRY'] == 'IT'].shape[0]
print(f"\nNumber of shipments from IT: {it_shipments_count}")


# Shipment count by country of arrival, only for those departing from IT
arrival_counts_it = data[data['DEPARTURE_COUNTRY'] == 'IT']['ARRIVAL_COUNTRY'].value_counts()
print("\nShipment count by country of arrival (departments from IT only):")
print(arrival_counts_it)


# Total shipments from IT to US
it_to_us_count = data[(data['DEPARTURE_COUNTRY'] == 'IT') & (data['ARRIVAL_COUNTRY'] == 'US')].shape[0]
print(f"Number of shipments from IT to US: {it_to_us_count}")


# Total shipments from IT to IT
it_to_it_count = data[(data['DEPARTURE_COUNTRY'] == 'IT') & (data['ARRIVAL_COUNTRY'] == 'IT')].shape[0]
print(f"Number of shipments from IT to IT: {it_to_it_count}")

Here we created some graphs in order to visualize, not extremely accurately, the distribution of the different type of data.
Than we cleaned the dataset removing the empty rows and features

In [ ]:
# List of specific numeric columns to analyze
colonne_numeriche_selezionate = [
    'GROSS_WEIGHT_KG',
    'NET_WEIGHT_KG',
    'VOLUME_M3',
    'DECLARED_DISTANCE_KM',
    'DELIVERY_TIME_HH',
    'WDAY'
]

# Create histograms for selected columns only
plt.figure(figsize=(20, 20))
data[colonne_numeriche_selezionate].hist(bins)
plt.suptitle("Distribution of the selected numerical variables")
plt.tight_layout()  # Automatically adjust gaps between subplots
plt.show()

In [ ]:
# Filter rows without null values
data_no_nulls = data.dropna()

# Save the new DataFrame to a CSV file
output_file_path = './delivery_clean.csv'
data_no_nulls.to_csv(output_file_path, index=False)

print(f"New CSV created with rows without null values: {output_file_path}")

# Prediction of missing values

In [ ]:
# Upload the CSV file
df = pd.read_csv("./delivery_clean.csv")

# Normalize column names
df.columns = df.columns.str.strip().str.upper()

We created a function to update the vehicle type base on the service type

In [ ]:
# Function to update VEHICLETYPE based on SERVICETYPE
def update_vehicletype(row):
    if pd.isna(row['VEHICLETYPE']) or row['VEHICLETYPE'] not in ['express', 'standard']:
        if 'corriere espresso' in str(row['SERVICETYPE']).lower():
            return 'express'
        else:
            return 'standard'
    return row['VEHICLETYPE']

# Apply function on 'VEHICLETYPE' column
df['VEHICLETYPE'] = df.apply(update_vehicletype, axis=1)

# Save the updated file
df.to_csv("./filtered_delivery_data.csv", index=False)

print("Updated file saved as 'filtered_delivery_data.csv'")

Than search the unique type of each feature and upload them in the csv

In [ ]:
# Find unique types and their count
unique_vehicle_types = df['VEHICLETYPE'].nunique()
all_vehicle_types = df['VEHICLETYPE'].unique()

print(f"Number of different types of vehicles: {unique_vehicle_types}")
print(f"Types of vehicles present: {all_vehicle_types}")

# Load the dataset
df = pd.read_csv("./delivery_data.csv")

Here we searched for the specific charateristics and then defined a function that let us predict the missing values

In [ ]:
# Select numerical characteristics for prediction
features = ['GROSS_WEIGHT_KG', 'NET_WEIGHT_KG', 'VOLUME_M3', 'DECLARED_DISTANCE_KM', 'DELIVERY_TIME_HH']

def predict_missing_values(df, target_column):
    print(f"\n{'='*50}")
    print(f"Predictive analytics for: {target_column}")
    print('='*50)
    
    df_known = df.dropna(subset=[target_column])
    df_unknown = df[df[target_column].isna()]
    
    if df_known.empty or df_unknown.empty:
        print(f"No value to predict for {target_column}")
        return df
    
    X = df_known[features]
    y = df_known[target_column].astype('category').cat.codes
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\nAccuracy metrics for {target_column}:")
    print(f"Overall accuracy: {accuracy:.4f}")
    
    print("\nDetailed classification report:")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': features,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nImportance of features:")
    for idx, row in feature_importance.iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")
    
    # Predicting missing values
    df_unknown[target_column] = model.predict(df_unknown[features])
    df = pd.concat([df_known, df_unknown])
    
    print(f"\nNumber of predicted values: {len(df_unknown)}")
    
    return df

Than we execute the predictions and upload them on a new dataframe

In [ ]:
# Run predictions
print("Starting predictive analytics...")
df = predict_missing_values(df, 'VEHICLETYPE')
df = predict_missing_values(df, 'SERVICETYPE')
print("\nRegression completed successfully!")

# Data cleanup

Here we upload the file obtained from the Prediction code

In [ ]:
# Upload the original CSV
df_originale = pd.read_csv('./filtered_delivery_data.csv')

print(df_originale.columns)
df_originale.columns = df_originale.columns.str.strip

Than we extracted the month from the shipping date and calculated the amount of shippings in each month, after that we calculated the amount of times each service type appears

In [ ]:
# Suppose the column is called 'SHIPPING_DATE'
# Extract only the month (6th and 7th characters)
df_originale['SHIPPING_DATE'] = df_originale['SHIPPING_DATE'].str[5:7]

# Print the result to check
print(df_originale['SHIPPING_DATE'])

# Count how many times it appears each month
conteggio_mesi = df_originale['SHIPPING_DATE'].value_counts()

# Prints the month count
print(conteggio_mesi)

# Count how many times each type of SERVICETYPE appears
conteggio_service_type = df_originale['SERVICETYPE'].value_counts()

Here we create the graph to show the spread of each service type

In [ ]:
# Create the pie chart
plt.figure(figsize=(8, 6))
plt.pie(conteggio_service_type, labels=conteggio_service_type.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of SERVICETYPE types')
plt.axis('equal')  # To maintain the circular shape
plt.show()

We count the amount of times a zipcode appears and then we calculate the percentage of each zipcode.
After this we filter the zipcode with a 2 or less percent of appearence and group them under the "others" definition

In [ ]:
# Count how many times each zip code appears in the 'DEPARTURE_ZIPCODE' column
conteggio_departure_zipcode = df_originale['DEPARTURE_ZIPCODE'].value_counts()

# Calculate the percentage for each zip code
percentuale_zipcode = conteggio_departure_zipcode / conteggio_departure_zipcode.sum() * 100

# Filter zip codes with percentage greater than 2%
zipcodes_frequenti = percentuale_zipcode[percentuale_zipcode > 2]

# Groups all other postcodes into the "Other" category
other = percentuale_zipcode[percentuale_zipcode <= 2].sum()
zipcodes_frequenti['Others'] = other

We created a pie graph to rappresent the diffusion of each zipcode in order to understand where was the primary start point 

In [ ]:
# Create the pie chart
plt.figure(figsize=(8, 6))
plt.pie(zipcodes_frequenti, labels=zipcodes_frequenti.index, autopct='%1.1f%%', startangle=140)
plt.title('Departure ZIP Code Distribution (DEPARTURE_ZIPCODE) - Greater than 2%')
plt.axis('equal')  # To maintain the circular shape
plt.show()

df = pd.read_csv('./filtered_delivery_data.csv')

# Find unique types and their count
unique_vehicle_types = df_originale['VEHICLETYPE'].nunique()
all_vehicle_types = df_originale['VEHICLETYPE'].unique()

print(f"Number of different types of vehicles: {unique_vehicle_types}")
print(f"Types of vehicles present: {all_vehicle_types}")

## Conclusions

We found out that there is a huge amount of deliveries that comes and goes from Italy compared to the amount from the US and there are none from Italy to the US.
We also noticed that more deliveries starts from the first days of the week, we recommend to try to analyze this phenomenom in order to understand and project a optimized way to spread those deliveries in the rest of the week.
Another thing we saw it's that most of the deliveries comes from a specific zipcode.